# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# read in weather py 
data_file = "../WeatherPy/Figures/WeatherPy_output.csv"
city_data_df = pd.read_csv(data_file)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Avarua,-21.21,-159.78,75.2,83,100,8.05,CK,1604195735
1,Mariehamn,60.10,19.93,48.2,87,0,16.11,AX,1604195736
2,Port Elizabeth,-33.92,25.57,64.4,82,75,8.05,ZA,1604195736
3,Korla,41.76,86.15,48.0,72,0,5.01,CN,1604195736
4,Punta Arenas,-53.15,-70.92,46.4,81,0,3.36,CL,1604195736


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# configer gmaps with API key
gmaps.configure(api_key=g_key)

# set variable for lat lng as locations and humidity to use as weight
locations = city_data_df[["Lat","Lng"]].astype(float)
humidity = city_data_df["Humidity"].astype(int)

In [8]:
# create humidity heatmap layer
fig=gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,
                                max_intensity = 100, point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
nice_weather_df = city_data_df.loc[(city_data_df["Max Temp"]>=70) & (city_data_df["Max Temp"]<=90)]
nice_weather_df = nice_weather_df.loc[nice_weather_df["Wind Speed"]<10.00]
nice_weather_df = nice_weather_df.loc[nice_weather_df["Cloudiness"]==0]
nice_weather_df.reset_index(inplace=True)
nice_weather_df.dropna()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,11,Teguise,29.06,-13.56,73.40,46,0,9.17,ES,1604195737
1,14,Rikitea,-23.12,-134.97,76.14,67,0,8.08,PF,1604195548
2,39,Airai,-8.93,125.41,78.10,35,0,3.27,TL,1604195555
3,56,Manga,11.66,-1.07,76.41,29,0,2.51,BF,1604195743
4,84,Ugoofaaru,5.67,73.00,82.20,68,0,2.93,MV,1604195747
5,108,Bundaberg,-24.85,152.35,88.00,33,0,9.17,AU,1604195572
6,154,Koungou,-12.73,45.20,73.40,88,0,2.24,YT,1604195758
7,179,Atambua,-9.11,124.89,87.35,41,0,4.99,ID,1604195762
8,196,Aswān,24.09,32.91,71.26,60,0,5.88,EG,1604195764
9,200,Chabahar,25.29,60.64,71.85,32,0,5.17,IR,1604195765


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:

# set target radius to 5000m
target_radius = 50000

# set type as hotel
target_type="hotel"

# formulate base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [14]:
# create lists to collect hotel data
hotel_list = []
# create for loop to collect hotel names list
for places in range(len(nice_weather_df)):
    lat = nice_weather_df.loc[places]['Lat']
    lng = nice_weather_df.loc[places]['Lng']
    params = {"location":f"{lat},{lng}" ,
             "radius": target_radius,
              "type":target_type,
              "key":g_key}
    requested = requests.get(base_url, params=params)
    hotel_list.append(requested.json()['results'][0]['name'])


In [23]:
# add hotel_list to nice weather df
# these are your vacation spots
nice_weather_df["Hotel Name"]=hotel_list
my_vacation_spots_df = nice_weather_df[["Hotel Name","Lat","Lng","Country","City"]]

In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = my_vacation_spots_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
hotel_name = my_vacation_spots_df["Hotel Name"]
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))